In [ ]:
import numpy as np
import torch
from pathlib import Path
from perception_bev_learning.utils import load_pkl
from os.path import join

EXPERIMENT_FOLDER = "/Data/Results/bev_learning/debug/train/full_testing"


pkl_files = [str(s) for s in Path(EXPERIMENT_FOLDER).glob("*.pkl")]

# LOAD DATA
hdd = {s.split("__")[-1][:-4]: load_pkl(s) for s in pkl_files if s.find("hdd") != -1}
layer_statistics = {
    s.split("__")[-1][:-4]: load_pkl(s) for s in pkl_files if s.find("statistic_reverted_layer_scaling") != -1
}
metrics = {s.split("__")[-1][:-4]: load_pkl(s) for s in pkl_files if s.find("meter_results") != -1}

In [ ]:
MODE = "test"
metrics[MODE].keys()

[k for k in layer_statistics["test"].keys() if k.find("obser") != -1]

In [ ]:
from prettytable import PrettyTable

x = PrettyTable()
x.field_names = ["Metric", "Value"]
for k, v in metrics[MODE].items():
    if k.find("_step") == -1:
        x.add_row([k, round(v, 4)])
print(x)

In [ ]:
from prettytable import PrettyTable
import pandas as pd

# Create a DataFrame from the list of dictionaries


data = {"name": [], "mode": [], "metric": [], "layer": [], "value": []}
for MODE in ["train", "test", "val"]:
    for k, v in metrics[MODE].items():
        if k.find("step") != -1:
            continue
        i1 = k.find("_pred")
        if i1 == -1:
            i1 = k.find("_current")

        i2 = k.find("__")

        data["layer"].append(k[5:i1])
        data["name"].append(k[i1 + 1 : i2])
        data["value"].append(v)
        data["mode"].append(MODE)
        data["metric"].append(k[i2 + 2 : -6])

df = pd.DataFrame.from_dict(data)
df[(df["mode"] == "test") * (df["layer"] == "elevation") * (df["metric"] == "mae")]
metrics[MODE]

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
for MODE in ["train", "val", "test"]:

    # Filter metrics based on keywords
    filtered_elevation_metrics = {}
    filtered_wheel_risk_cvar_metrics = {}

    for k, v in metrics[MODE].items():
        if "pred_target" in k or "current_target" in k:
            if ("_mae_" in k or "_wmae_" in k) and "elevation" in k:
                if "_epoch" in k:
                    filtered_elevation_metrics[k] = v
            elif ("_mse_" in k or "_wmse_" in k) and "wheel_risk_cvar" in k:
                if "_epoch" in k:
                    filtered_wheel_risk_cvar_metrics[k] = v

    # Extract metric names and values for plotting
    elevation_metric_names = list(filtered_elevation_metrics.keys())
    elevation_metric_values = list(filtered_elevation_metrics.values())

    wheel_risk_cvar_metric_names = list(filtered_wheel_risk_cvar_metrics.keys())
    wheel_risk_cvar_metric_values = list(filtered_wheel_risk_cvar_metrics.values())

    # Create a figure with subplots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

    # Bar plot for elevation metrics
    elevation_bar_colors = ["blue" if "current_target" in metric else "orange" for metric in elevation_metric_names]
    axes[0].bar(
        range(len(elevation_metric_names)),
        elevation_metric_values,
        align="center",
        alpha=0.7,
        color=elevation_bar_colors,
    )
    axes[0].set_xticks(range(len(elevation_metric_names)))
    axes[0].set_xticklabels(elevation_metric_names, rotation=0)
    axes[0].set_xlabel("Metric")
    axes[0].set_ylabel("Value")
    axes[0].set_title("Elevation Metrics")
    axes[0].grid(True)
    axes[0].tick_params(axis="x", rotation=45)

    # Bar plot for wheel risk cvar metrics
    wheel_risk_cvar_bar_colors = [
        "blue" if "current_target" in metric else "orange" for metric in wheel_risk_cvar_metric_names
    ]
    axes[1].bar(
        range(len(wheel_risk_cvar_metric_names)),
        wheel_risk_cvar_metric_values,
        align="center",
        alpha=0.7,
        color=wheel_risk_cvar_bar_colors,
    )
    axes[1].set_xticks(range(len(wheel_risk_cvar_metric_names)))
    axes[1].set_xticklabels(wheel_risk_cvar_metric_names, rotation=0)
    axes[1].set_xlabel("Metric")
    axes[1].set_ylabel("Value")
    axes[1].set_title("Wheel Risk CVaR Metrics")
    axes[1].grid(True)
    axes[1].tick_params(axis="x", rotation=45)

    fig.suptitle(f"Metrics for MODE: {MODE}", fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()

In [ ]:
from perception_bev_learning.dataset import get_bev_dataloader
from perception_bev_learning.cfg import ExperimentParams
from perception_bev_learning.utils import denormalize_img
from perception_bev_learning.visu import LearningVisualizer, show
from perception_bev_learning.visu import show, get_img_from_fig
from pytorch_lightning import seed_everything
import matplotlib.pyplot as plt
from perception_bev_learning.visu import paper_colors_rgb_f

LAYER = "wheel_risk_cvar"
KEY = "bev_metric"

METRICS = ["pred_target__cell_stat", "current_target__cell_stat"]
for MODE in ["train", "val", "test"]:  # , "val", "train"]:
    fig, axes = plt.subplots(1, len(METRICS), sharey=True, figsize=(10, 5))  # Create subplots
    print(LAYER)
    for j, METRIC in enumerate(METRICS):
        # stat = "pred_statistics"
        # stat = "current_statistics"

        recall = layer_statistics[MODE][f"{METRIC}_{KEY}_{LAYER}_test_recall_bins"]
        f1 = layer_statistics[MODE][f"{METRIC}_{KEY}_{LAYER}_test_f1_bins"]
        precision = layer_statistics[MODE][f"{METRIC}_{KEY}_{LAYER}_test_precision_bins"]
        x_values = np.arange(0, len(f1)) + 1
        # Create a figure and a set of subplots

        # Plot hazard_detection_ratio
        axes[j].plot(x_values, [r[1] for r in recall], label="Recall", color=list(paper_colors_rgb_f["orange"]))
        # Plot false_hazard_ratio
        axes[j].plot(x_values, [r[1] for r in f1], label="F1", color=list(paper_colors_rgb_f["mangenta"]))
        axes[j].plot(x_values, [r[1] for r in precision], label="Precision", color=list(paper_colors_rgb_f["blue"]))

        # Set labels and title
        axes[j].set_xlabel("Distance in m")
        axes[j].set_ylabel("Metric")
        axes[j].set_title(f"{MODE} - {METRIC} Binning Results")
        axes[j].legend()

    # Display the plot
    plt.tight_layout()  # Adjust layout to prevent overlapping
    plt.show()

In [ ]:
from perception_bev_learning.dataset import get_bev_dataloader
from perception_bev_learning.cfg import ExperimentParams
from perception_bev_learning.utils import denormalize_img
from perception_bev_learning.visu import LearningVisualizer, show
from perception_bev_learning.visu import show, get_img_from_fig
from pytorch_lightning import seed_everything
import matplotlib.pyplot as plt
from perception_bev_learning.visu import paper_colors_rgb_f

LAYER = "elevation"
KEY = "bev_metric"

METRICS = ["pred_target__observed_mae", "current_target__observed_mae"]
for MODE in ["train"]:  # , "val", "train"]: "train", "val",
    fig, axes = plt.subplots(1, len(METRICS), sharey=True, figsize=(10, 5))  # Create subplots
    print(LAYER)
    for j, METRIC in enumerate(METRICS):
        # stat = "pred_statistics"
        # stat = "current_statistics"
        # print("current_target__unobserved_mse_bev_metric_wheel_risk_cvar_test")
        print(f"{METRIC}:")
        print(layer_statistics[MODE][f"{METRIC}_{KEY}_{LAYER}_test"])

In [ ]:
from perception_bev_learning.dataset import get_bev_dataloader
from perception_bev_learning.cfg import ExperimentParams
from perception_bev_learning.utils import denormalize_img
from perception_bev_learning.visu import LearningVisualizer, show
from perception_bev_learning.visu import show, get_img_from_fig
from pytorch_lightning import seed_everything
import matplotlib.pyplot as plt
from perception_bev_learning.visu import paper_colors_rgb_f


METRICS = ["pred_target__cell_stat", "current_target__cell_stat"]
cut_off = 50
for MODE in ["train", "val", "test"]:  # , "val", "train"]:

    fig, axes = plt.subplots(1, 4, sharey=False, figsize=(14, 3), constrained_layout=False)  # Create subplots

    for k, value in enumerate(["precision", "recall", "f1"]):
        for res in zip(["mangenta", "orange"], METRICS, ["RoadRunner", "RACER-X"]):
            if k < 3:

                color, METRIC, axes_label = res
                LAYER = "wheel_risk_cvar"
                KEY = "bev_metric"

                met = layer_statistics[MODE][f"{METRIC}_{KEY}_{LAYER}_test_{value}_bins"]

                x_values = np.arange(0, cut_off) + 1
                # Create a figure and a set of subplots

                # Plot hazard_detection_ratio
                axes[k].plot(
                    x_values, [r[1] for r in met][:cut_off], label=axes_label, color=list(paper_colors_rgb_f[color])
                )

                # Set labels and title
                axes[k].set_xlabel("Distance in [m]")
                axes[k].set_ylabel(value.capitalize())
                axes[k].set_title(f"Hazard Detection: {value.capitalize()}")
                axes[k].legend()
                axes[k].legend(fontsize=10)
                # axes[k].spines['top'].set_visible(False)
                # axes[k].spines['right'].set_visible(False)

                axes[k].set_ylim(bottom=0)
                axes[k].set_ylim(top=1)
                axes[k].set_xlim(left=0)

    # MSE plot
    METRIC_CUR = "current_target__cell_stat"
    METRIC_TAR = "pred_target__cell_stat"
    mean_tar = layer_statistics[MODE][f"{METRIC_TAR}_{KEY}_{LAYER}_test_mean_bins"]
    mean_cur = layer_statistics[MODE][f"{METRIC_CUR}_{KEY}_{LAYER}_test_mean_bins"]

    # Plot hazard_detection_ratio
    axes[3].plot(x_values, mean_tar[:cut_off], label=f"RoadRunner", color=list(paper_colors_rgb_f["mangenta"]))
    axes[3].plot(x_values, mean_cur[:cut_off], label=f"RACER-X", color=list(paper_colors_rgb_f["orange"]))

    # Set labels and title
    axes[3].set_xlabel("Distance in [m]", fontsize=12)
    axes[3].set_ylabel("MSE", fontsize=12)
    axes[3].set_title(f"Layer {LAYER}: MSE")
    axes[3].legend(fontsize=10)
    axes[3].grid(True)
    axes[3].set_ylim(bottom=0)

    axes[3].set_ylim(top=0.2)
    axes[3].set_xlim(left=0)

    # axes[1].set_yticklabels([])
    # axes[2].set_yticklabels([])
    # plt.subplots_adjust(wspace=0.0)
    # Display the plot
    # print(MODE)
    plt.tight_layout()  # Adjust layout to prevent overlapping
    plt.savefig(join(EXPERIMENT_FOLDER, f"hazard_detection_metric_{MODE}.svg"), format="svg")
    plt.savefig(join(EXPERIMENT_FOLDER, f"hazard_detection_metric_{MODE}.pdf"), format="pdf")
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

for res in zip(["wheel_risk_cvar", "elevation"], ["MSE in [m2]", "MAE in [m]"]):
    LAYER, y_label = res
    MODES = ["train", "val", "test"]
    KEY = "bev_metric"
    METRIC_CUR = "current_target__cell_stat"
    METRIC_TAR = "pred_target__cell_stat"

    fig, axes = plt.subplots(1, len(MODES), sharey=True, figsize=(10, 3))  # Create subplots

    for idx, MODE in enumerate(MODES):
        mean_tar = layer_statistics[MODE][f"{METRIC_TAR}_{KEY}_{LAYER}_test_mean_bins"]
        mean_cur = layer_statistics[MODE][f"{METRIC_CUR}_{KEY}_{LAYER}_test_mean_bins"]

        x_values = np.arange(0, len(mean_cur)) + 1

        # Plot hazard_detection_ratio
        axes[idx].plot(x_values, mean_tar, label=f"RoadRunner", color=list(paper_colors_rgb_f["mangenta"]))
        axes[idx].plot(x_values, mean_cur, label=f"RACER-X", color=list(paper_colors_rgb_f["orange"]))

        # Set labels and title
        axes[idx].set_xlabel("Distance in [m]", fontsize=12)
        axes[idx].set_ylabel(y_label, fontsize=12)
        axes[idx].set_title(f"Layer: {LAYER} - Mode: {MODE}")
        axes[idx].legend(fontsize=12)
        axes[idx].grid(True)
        axes[idx].set_ylim(bottom=0)
        if LAYER == "wheel_risk_cvar":
            axes[idx].set_ylim(top=0.2)
        else:
            axes[idx].set_ylim(top=2)
        axes[idx].set_xlim(left=0)

        # axes[idx].spines['top'].set_visible(False)
        # axes[idx].spines['right'].set_visible(False)

    plt.tight_layout()  # Adjust layout to prevent overlapping
    plt.savefig(join(EXPERIMENT_FOLDER, f"{LAYER}_distance.svg"), format="svg")
    plt.savefig(join(EXPERIMENT_FOLDER, f"{LAYER}_distance.pdf"), format="pdf")
    plt.show()

In [ ]:
print(MODE)
MODE = "test"
for MODE in ["test", "train"]:
    layer_statistics[MODE].keys()

    visu = LearningVisualizer(p_visu=EXPERIMENT_FOLDER)

    i1 = visu.plot_elevation_map_two(
        layer_statistics[MODE]["current_target__cell_stat_bev_metric_wheel_risk_cvar_test_mean_map"],
        layer_statistics[MODE]["pred_target__cell_stat_bev_metric_wheel_risk_cvar_test_mean_map"],
        tag=f"{MODE}_epoch_statistic_mae_current_pred_wheel_risk",
        cmap_name="turbo",
        v_max=1,
        v_min=0,
        labels=["RACER-X", "RoadRunner"],
        title="MSE - wheel_risk_cvar",
        store_svg=True,
    )

    def max_nan(t):
        m = ~torch.isnan(t)
        return t[m].max()

    current_mean = layer_statistics[MODE]["current_target__cell_stat_bev_metric_elevation_test_mean_map"]
    pred_mean = layer_statistics[MODE]["pred_target__cell_stat_bev_metric_elevation_test_mean_map"]
    ma = round(float(max(max_nan(current_mean), max_nan(pred_mean))), 3)

    i2 = visu.plot_elevation_map_two(
        current_mean,
        pred_mean,
        tag=f"{MODE}_epoch_statistic_mae_current_pred_elevation",
        cmap_name="turbo",
        v_max=2.5,
        v_min=0,
        labels=["RACER-X", "RoadRunner"],
        title="MSE - wheel_risk_cvar",
        subtitle=False,
        store_svg=True,
    )
    show(i1, title=f"{MODE} - Wheel_Risk"), show(i2, title=f"{MODE} -Elevation")

In [ ]:
[k for k in layer_statistics[MODE].keys() if k.find("mean") != -1]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metric = "elevation"
for MODE in ["test", "train"]:
    # 72meter is the max distance consired
    current_mean_bins = layer_statistics[MODE][f"{metric}_current_mean_bins"][:50]
    pred_mean_bins = layer_statistics[MODE][f"{metric}_pred_mean_bins"][:50]

    fig, axs = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    fig.suptitle(f"{MODE.capitalize()} Data - {metric}")

    # Plot current bins
    axs[0].bar(np.arange(len(current_mean_bins)), current_mean_bins * 20, color="blue", label="Current Bins")
    axs[0].set_xlabel("Bins in meter")
    axs[0].set_ylabel("Values")
    axs[0].set_title("Racer-X Elevation MAE")
    axs[0].legend()

    # Plot predicted bins
    axs[1].bar(np.arange(len(pred_mean_bins)), pred_mean_bins * 20, color="red", label="Pred Bins")
    axs[1].set_xlabel("Bins in meter")
    axs[1].set_ylabel("Values")
    axs[1].set_title("RoadRunner Elevation MAE")
    axs[1].legend()
    plt.tight_layout()
    plt.show()